[2024-08-09 Fiddler](https://thefiddler.substack.com/p/can-you-hack-gymnastics)
====================


[Simulations](20240809.go) gives about 96.2% and, for the extra credit, 91.6%.

    $ go run 20240809.go
    9617624/10000000 0.961762
    9167039/10000000 0.916704

Setting things up

In [1]:
%display latex
D_A, X_A, D_B, X_B = var('D_A X_A D_B X_B')
A_s = D_A + X_A
A_p = D_A * X_A
B_s = D_B + X_B
B_p = D_B * X_B

The probability that the relative rankings are the same is

$$ \int_0^{10} \frac{dX_A}{10} \int_0^{10} \frac{dX_B}{10} (H(A_s-B_s)H(A_p-B_p) + H(B_s-A_s)H(B_p-A_p)) $$

but I can't figure out how to express that without getting `TypeError`s.  The integrand is

In [2]:
heaviside(A_s-B_s)*heaviside(A_p-B_p) + heaviside(B_s-A_s)*heaviside(B_p-A_p)

heaviside(D_A*X_A - D_B*X_B)*heaviside(D_A - D_B + X_A - X_B) + heaviside(-D_A*X_A + D_B*X_B)*heaviside(-D_A + D_B - X_A + X_B)

A is ahead of B when adding when $X_B < D_A - D_B + X_A$.

A is ahead of B when multiplying when $X_B < D_AX_A/D_B$.

So the relative rankings are the same when $X_B < \min(D_A-D_B+X_A,D_AX_A/D_B)$ or $X_B > \max(D_A-D_B+X_A,D_AX_A/D_B)$.

In [3]:
solve(D_A-D_B+X_A == D_A*X_A/D_B, X_A)

[X_A == D_B]

Assuming $D_A > D_B$, then when $X_A < D_B$, the relative rankings are the same when
$X_B < D_AX_A/D_B$ or $X_B > D_A-D_B+X_A$, and when $X_A > D_B$, the relative rankings
are them same when $X_B < D_A-D_B+X_A$ or $X_B > D_AX_A/D_B$.

If $X_A < D_B$, the probability that $X_B < D_AX_A/D_B$ is $D_AX_A/D_B\cdot1/10$, and
the probability that $X_B > D_A-D_B+X_A$ is 0 when $D_A-D_B+X_A > 10$, or $X_A > 10 - (D_A-D_B)$,
and $(10 - (D_A-D_B+X_A))/10$ when $X_A < 10-(D_A-D_B)$.

If $X_A > D_B$, the probability that $X_B < D_A-D_B+X_A$ is $\min(1,(D_A-D_B+X_A)/10)$, and
the probability that $X_B > D_AX_A/D_B$ is 0 when $D_AX_A/D_B > 10$, or $X_A > 10D_B/D_A$,
is $\max(0,(10 - D_AX_A/D_B)/10)$.

The probability that the relative rankings are the same is

In [4]:
p(D_A,D_B) = integral(D_A*X_A/D_B*1/10, X_A, 0, D_B)/10 \
    + integral((10 - (D_A-D_B+X_A))/10, X_A, 0, D_B)/10 \
    + integral((D_A-D_B+X_A)/10, X_A, D_B, 10-(D_A-D_B))/10 \
    + integral(1, X_A, 10-(D_A-D_B), 10)/10 \
    + integral((10-D_A*X_A/D_B)/10, X_A, D_B, 10*D_B/D_A)/10
p

(D_A, D_B) |--> -1/200*D_A^2 - 1/100*(D_A - 10)*D_B + 1/200*(D_A - 20)*D_B + 1/200*D_A*D_B + 1/200*D_B^2 + 1/10*D_A - 1/10*D_B + 1/2*D_B/D_A + 1/2

In [5]:
(p(6,5), numerical_approx(p(6,5)))

(577/600, 0.961666666666667)

That agrees reasonably with the results of the simulations.

Extra credit
------------
Integrating over $D_A$ and $D_B$, assuming $D_A > D_B$, then doubling to count the $D_A < D_B$ cases,

In [6]:
extra_credit = 2*integral(integral(p, D_B, 0, D_A)/10, D_A, 0, 10)/10
(extra_credit, numerical_approx(extra_credit))

(11/12, 0.916666666666667)

This also reasonably agrees with the results of the simulations.

Making the rounds
-----------------
If an $n$-dimensional ball of radius $r$ has volume $A_nr^n$, then an $n+1$-dimensional ball will be made up
of $n$-dimensional balls of radius $\rho_n$ that are centered $x_n$ from the origin along the axis in the
$n+1$th dimension, where $\rho_n^2 + x_n^2 = 1$, so $\rho_n = \sqrt{1 - x_n^2}$.  The volume is then

$$
    \int_{-1}^1 dx\, A_n\rho^n = 2\int_0^1 dx\, A_n (1-x^2)^{n/2}
$$


In [7]:
n = var('n')
assume(n > 0)
f(n) = 2*integral((1-x^2)^(n/2), x, 0, 1)
f

n |--> beta(1/2, 1/2*n + 1)

In [8]:
m = var('m')
A(n) = product(f(m), m, 0, n-1)
A

n |--> product(beta(1/2, 1/2*j1 + 1/2), j1, 1, n)

In [9]:
r = var('r')
[A(i).simplify()*r^i for i in [0..15]]

[1,
 2*r,
 pi*r^2,
 4/3*pi*r^3,
 1/2*pi^2*r^4,
 8/15*pi^2*r^5,
 1/6*pi^3*r^6,
 16/105*pi^3*r^7,
 1/24*pi^4*r^8,
 32/945*pi^4*r^9,
 1/120*pi^5*r^10,
 64/10395*pi^5*r^11,
 1/720*pi^6*r^12,
 128/135135*pi^6*r^13,
 1/5040*pi^7*r^14,
 256/2027025*pi^7*r^15]

In [10]:
[numerical_approx(A(i).simplify(),digits=3)*r^i for i in [0..15]]

[1.00,
 2.00*r,
 3.14*r^2,
 4.19*r^3,
 4.94*r^4,
 5.26*r^5,
 5.17*r^6,
 4.72*r^7,
 4.06*r^8,
 3.30*r^9,
 2.55*r^10,
 1.88*r^11,
 1.34*r^12,
 0.911*r^13,
 0.599*r^14,
 0.381*r^15]

Since $A_{n+1} = A_n \text{B}\left(\frac{1}{2},\frac{n+2}{2}\right)$, and $\text{B}(x,y) \sim \Gamma(x) y^{-x}$
for large $y$, then

$$
    A_{n+1} \sim A_n \Gamma\left(\frac12\right) \sqrt{\frac{2}{n+2}} = A_n\sqrt{\frac{2\pi}{n+2}}
$$

so the volume of a unit ball goes to zero for large $n$, and the maximum is $8\pi^2/15$ at $n = 5$, which
makes sense because $\frac{2\pi}{n+2} > 1$ for $n < 5$ and $\frac{2\pi}{n+2} < 1$ for $n \ge 5$.

In [11]:
[(numerical_approx(A(i+1).simplify(),digits=3)*r^(i+1),numerical_approx(A(i).simplify()*sqrt(2*pi/(i+2)),digits=3)*r^(i+1)) for i in [10..20]]

[(1.88*r^11, 1.85*r^11),
 (1.34*r^12, 1.31*r^12),
 (0.911*r^13, 0.895*r^13),
 (0.599*r^14, 0.589*r^14),
 (0.381*r^15, 0.376*r^15),
 (0.235*r^16, 0.232*r^16),
 (0.141*r^17, 0.139*r^17),
 (0.0822*r^18, 0.0811*r^18),
 (0.0466*r^19, 0.0460*r^19),
 (0.0258*r^20, 0.0255*r^20),
 (0.0139*r^21, 0.0138*r^21)]